# Data exploration :

In [1]:
import pandas as pd
ames_housing = pd.read_csv("../datasets/ames_housing_no_missing.csv")

target_name = "SalePrice"
data, target = ames_housing.drop(columns=target_name), ames_housing[target_name]
target = (target > 200_000).astype(int)

In [2]:
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,Grvl,Reg,Lvl,AllPub,Inside,...,0,0,Gd,MnPrv,Shed,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,Grvl,Reg,Lvl,AllPub,FR2,...,0,0,Gd,MnPrv,Shed,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,Grvl,IR1,Lvl,AllPub,Inside,...,0,0,Gd,MnPrv,Shed,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,Grvl,IR1,Lvl,AllPub,Corner,...,0,0,Gd,MnPrv,Shed,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,Grvl,IR1,Lvl,AllPub,FR2,...,0,0,Gd,MnPrv,Shed,0,12,2008,WD,Normal


In [3]:
data.dtypes.value_counts()

object     43
int64      33
float64     3
dtype: int64

# Only numeric data :

In [4]:
numerical_features = [
  "LotFrontage", "LotArea", "MasVnrArea", "BsmtFinSF1", "BsmtFinSF2",
  "BsmtUnfSF", "TotalBsmtSF", "1stFlrSF", "2ndFlrSF", "LowQualFinSF",
  "GrLivArea", "BedroomAbvGr", "KitchenAbvGr", "TotRmsAbvGrd", "Fireplaces",
  "GarageCars", "GarageArea", "WoodDeckSF", "OpenPorchSF", "EnclosedPorch",
  "3SsnPorch", "ScreenPorch", "PoolArea", "MiscVal",
]

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate

data_numeric = data[numerical_features]

model = make_pipeline(StandardScaler(), LogisticRegression(max_iter=500))

cv_result_categorical = cross_validate(model, data_numeric, target, cv=10)
cv_result_categorical

{'fit_time': array([0.03761196, 0.04914641, 0.04957104, 0.04802489, 0.04537821,
        0.04123044, 0.0399406 , 0.03979707, 0.04253769, 0.05155182]),
 'score_time': array([0.        , 0.        , 0.00806856, 0.        , 0.00805736,
        0.00798845, 0.00799632, 0.00846481, 0.        , 0.        ]),
 'test_score': array([0.9109589 , 0.89041096, 0.9109589 , 0.88356164, 0.90410959,
        0.88356164, 0.88356164, 0.87671233, 0.89726027, 0.87671233])}

In [6]:
scores = cv_result_categorical["test_score"]
print(f"The mean cross-validation accuracy is {scores.mean():.3f} +/- {scores.std():.3f}")

The mean cross-validation accuracy is 0.892 +/- 0.013


# Both numerical and categorical data :

In [7]:
categorical_features = [_ for _ in data.columns.tolist() if _ not in numerical_features]

In [8]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate

numerical_preprocessor = StandardScaler()
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore", sparse=False)
preprocessor = ColumnTransformer([
    ('numerical', numerical_preprocessor, numerical_features),
    ('categorical', categorical_preprocessor, categorical_features)
])

model = make_pipeline(preprocessor, LogisticRegression(max_iter=500))

cv_result_both = cross_validate(model, data, target, cv=10)
cv_result_both

{'fit_time': array([0.61815143, 0.50006104, 0.32715297, 0.33117676, 0.42596769,
        0.33125257, 0.37305713, 0.3636651 , 0.38853526, 0.42346716]),
 'score_time': array([0.03312993, 0.03202248, 0.02521515, 0.01839256, 0.00799251,
        0.0199635 , 0.02399039, 0.02548432, 0.02403426, 0.01597905]),
 'test_score': array([0.95890411, 0.90410959, 0.89041096, 0.92465753, 0.9109589 ,
        0.93835616, 0.90410959, 0.91780822, 0.92465753, 0.89726027])}

In [9]:
scores = cv_result_both["test_score"]
print(f"The mean cross-validation accuracy is {scores.mean():.3f} +/- {scores.std():.3f}")

The mean cross-validation accuracy is 0.917 +/- 0.019


# Comparing models :

In [10]:
cv_result_categorical["test_score"]

array([0.9109589 , 0.89041096, 0.9109589 , 0.88356164, 0.90410959,
       0.88356164, 0.88356164, 0.87671233, 0.89726027, 0.87671233])

In [11]:
cv_result_both["test_score"]

array([0.95890411, 0.90410959, 0.89041096, 0.92465753, 0.9109589 ,
       0.93835616, 0.90410959, 0.91780822, 0.92465753, 0.89726027])

In [12]:
(cv_result_both["test_score"] > cv_result_categorical["test_score"]).sum()

9

In [13]:
import sklearn
sklearn.__version__

'1.0.2'